In [1]:
from ortools.sat.python import cp_model

In [2]:
MAX_VALUE = 99999999

N = 0
M = 0
classes = []
room_capacity = []

def read_input(file_name):
    global N, M, classes, room_capacity

    with open(file_name) as file:
        lines = [line.rstrip() for line in file]

    N, M = tuple(int(x) for x in lines[0].split())

    for i in range(N):
        num_periods, teacher, num_pupils = tuple(int(x) for x in lines[i+1].split())
        classes.append({
            'num_periods': num_periods,
            'teacher': teacher,
            'num_pupils': num_pupils,
        })

    room_capacity = [int(x) for x in lines[-1].split()]

    if len(classes) != N or len(room_capacity) != M:
        raise Exception('Error when reading data!')

read_input('./data/input1.txt')

In [3]:
classes

[{'num_periods': 4, 'teacher': 16, 'num_pupils': 20},
 {'num_periods': 3, 'teacher': 11, 'num_pupils': 26},
 {'num_periods': 3, 'teacher': 18, 'num_pupils': 30},
 {'num_periods': 4, 'teacher': 16, 'num_pupils': 24},
 {'num_periods': 3, 'teacher': 7, 'num_pupils': 27},
 {'num_periods': 3, 'teacher': 15, 'num_pupils': 13},
 {'num_periods': 2, 'teacher': 15, 'num_pupils': 18},
 {'num_periods': 4, 'teacher': 20, 'num_pupils': 12},
 {'num_periods': 2, 'teacher': 13, 'num_pupils': 10},
 {'num_periods': 3, 'teacher': 20, 'num_pupils': 29},
 {'num_periods': 2, 'teacher': 18, 'num_pupils': 14},
 {'num_periods': 4, 'teacher': 6, 'num_pupils': 10},
 {'num_periods': 4, 'teacher': 11, 'num_pupils': 28},
 {'num_periods': 4, 'teacher': 8, 'num_pupils': 19},
 {'num_periods': 3, 'teacher': 4, 'num_pupils': 17},
 {'num_periods': 2, 'teacher': 20, 'num_pupils': 26},
 {'num_periods': 2, 'teacher': 1, 'num_pupils': 11},
 {'num_periods': 2, 'teacher': 19, 'num_pupils': 15},
 {'num_periods': 4, 'teacher': 9,

In [4]:
room_capacity

[28,
 35,
 29,
 28,
 27,
 25,
 16,
 15,
 23,
 29,
 33,
 35,
 18,
 31,
 33,
 19,
 27,
 30,
 25,
 19,
 19,
 15,
 16,
 31,
 35,
 23,
 25,
 21,
 21,
 29,
 24,
 31,
 20,
 28,
 24,
 31,
 33,
 28,
 33,
 23,
 31,
 34,
 20,
 18,
 23,
 28,
 17,
 15,
 35,
 29]

# MODEL

In [5]:
model = cp_model.CpModel()

In [6]:
# VARIABLES & DOMAINS
class_is_chosen = [] # Lớp có được xếp hay không, 1 nếu được xếp, 0 nếu bỏ
class_half_day = [] # Buổi học, D = {0, ..., 9} 
class_starting = [] # Tiết học bắt đầu (trong buổi), D = {0, ..., 5}
class_room = [] # Phòng học, D = {0, ..., M-1}

for i in range(N):
    class_is_chosen.append(model.NewBoolVar(f'class_is_chosen[{i}]'))
    class_half_day.append(model.NewIntVar(0, 9, f'class_half_day[{i}]'))
    class_starting.append(model.NewIntVar(0, 5, f'class_starting[{i}]'))
    class_room.append(model.NewIntVar(0, M-1, f'clas_room[{i}]'))

In [7]:
# CONSTRAINTS

for i in range(N):

    # Sĩ số lớp được chọn ít hơn kích thước phòng
    temp_room_capacity = model.NewIntVar(0, MAX_VALUE, f'temp_room_capacity[{i}]')
    model.AddElement(class_room[i], room_capacity, temp_room_capacity)
    model.Add(
        temp_room_capacity >= classes[i]['num_pupils']
    ).OnlyEnforceIf(class_is_chosen[i])

    # Lớp học giới hạn trong một buổi
    model.Add(
        class_starting[i] + classes[i]['num_periods'] <= 6
    ).OnlyEnforceIf(class_is_chosen[i])

for i in range(N):
    for j in range(N):
        if i == j:
            continue

        # Biến: hai lớp có chồng giờ không
        i_same_session_j = model.NewBoolVar(f'i_same_session_j[{i},{j}]')
        model.Add(class_half_day[i] == class_half_day[j]).OnlyEnforceIf(i_same_session_j)
        model.Add(class_half_day[i] != class_half_day[j]).OnlyEnforceIf(i_same_session_j.Not())

        i_start_before_j = model.NewBoolVar(f'i_start_before_j[{i},{j}]')
        model.Add(class_starting[i] <= class_starting[j]).OnlyEnforceIf(i_start_before_j)
        model.Add(class_starting[i] > class_starting[j]).OnlyEnforceIf(i_start_before_j.Not())

        i_end_after_j_start = model.NewBoolVar(f'i_end_after_j_start[{i},{j}]')
        model.Add(class_starting[i] + classes[i]['num_periods'] - 1 >= class_starting[j]).OnlyEnforceIf(i_end_after_j_start)
        model.Add(class_starting[i] + classes[i]['num_periods'] - 1 < class_starting[j]).OnlyEnforceIf(i_end_after_j_start.Not())

        overlap_requirements = sum([i_same_session_j, i_start_before_j, i_end_after_j_start])
        
        temp_overlap_1 = model.NewBoolVar(f'temp_overlap_1[{i},{j}]')
        model.AddBoolAnd([i_start_before_j, i_end_after_j_start]).OnlyEnforceIf(temp_overlap_1)
        model.AddBoolOr([i_start_before_j.Not(), i_end_after_j_start.Not()]).OnlyEnforceIf(temp_overlap_1.Not())

        i_overlapped_by_j = model.NewBoolVar(f'i_overlapped_by_j[{i},{j}]')
        model.AddBoolAnd([i_same_session_j, temp_overlap_1]).OnlyEnforceIf(i_overlapped_by_j)
        model.AddBoolOr([i_same_session_j.Not(), temp_overlap_1.Not()]).OnlyEnforceIf(i_overlapped_by_j.Not())


        # Hai lớp cùng giáo viên không chồng lên nhau
        if classes[i]['teacher'] == classes[j]['teacher']:
            model.Add(
                i_overlapped_by_j == 0
            ).OnlyEnforceIf(
                class_is_chosen[i] 
            ).OnlyEnforceIf(
                class_is_chosen[j]
            )

        # Hai lớp xếp chồng giờ không cùng phòng
        i_same_room_j = model.NewBoolVar(f'i_same_room_j[{i},{j}]')
        model.Add(class_room[i] == class_room[j]).OnlyEnforceIf(i_same_room_j)
        model.Add(class_room[i] != class_room[j]).OnlyEnforceIf(i_same_room_j.Not())

        model.Add(
            i_overlapped_by_j == 0
        ).OnlyEnforceIf(
            class_is_chosen[i] 
        ).OnlyEnforceIf(
            class_is_chosen[j]
        ).OnlyEnforceIf(
            i_same_room_j
        )

In [8]:
# OBJECTIVES
model.Maximize(
    sum(class_is_chosen)
)

In [9]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

MemoryError: bad allocation

In [ ]:
solution = {}

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for idx, c in enumerate(class_is_chosen):
        if solver.Value(c):
            solution[idx + 1] = {
                'starting': (
                    solver.Value(class_half_day[idx]) * 6 + solver.Value(class_starting[idx]) + 1
                ),
                'room': solver.Value(class_room[idx]) + 1,
            }
            
else:
    print('No solution found.')

In [ ]:
print(len(solution))
for i in solution:
    starting = solution[i]['starting']
    room = solution[i]['room']
    print(f'{i} {starting} {room}')

9
1 3 1
2 9 1
3 14 1
4 29 1
5 33 2
6 31 1
8 26 2
9 25 1
10 22 2
